In [83]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import ast
import numpy as np
import statsmodels.api as sm

In [69]:
# Main merged dataset
games = pd.read_csv("../data/merged_weekly_games_with_rankings.csv")
games = games.drop_duplicates(subset=["id"])

# External ratings data (Team 1 vs Team 2 format)
ratings = pd.read_csv("../data/ratings_2012_2022.csv")

# Crosswalk to map team names
crosswalk = pd.read_csv("../data/football_crosswalk.csv")


In [70]:
crosswalk.columns = ["cfbdata_name", "ratings_name"]

# lowercase and strip to match ratings style
crosswalk["ratings_name"] = crosswalk["ratings_name"].str.lower().str.replace(" ", "")


In [81]:
games.tail()

,id,season,week,seasonType,startDate,startTimeTBD,completed,neutralSite,conferenceGame,attendance,...,lines,homeWinProbability,homeRank,awayRank,homeRanked,awayRanked,rankedTeamCount,isRankedMatchup,ratings_home_team,ratings_away_team
12743,401677107,2024,1,postseason,2025-01-04T00:30:00.000Z,False,True,True,False,NaN,...,"[{'provider': 'DraftKings', 'spread': 10, 'for...",0.245,NaN,NaN,False,False,0,False,virginia-tech,minnesota
12744,401677108,2024,1,postseason,2025-01-04T16:00:00.000Z,False,True,True,False,NaN,...,"[{'provider': 'DraftKings', 'spread': 4.5, 'fo...",0.391,NaN,NaN,False,False,0,False,liberty,buffalo
12745,401677189,2024,1,postseason,2025-01-10T00:30:00.000Z,False,True,True,False,NaN,...,"[{'provider': 'DraftKings', 'spread': -1.5, 'f...",0.527,5.0,2.0,True,True,2,True,penn-state,notre-dame
12746,401677191,2024,1,postseason,2025-01-11T00:30:00.000Z,False,True,True,False,NaN,...,"[{'provider': 'DraftKings', 'spread': 6.5, 'fo...",0.352,4.0,1.0,True,True,2,True,texas,ohio-state
12747,401677192,2024,1,postseason,2025-01-21T00:30:00.000Z,False,True,True,False,NaN,...,"[{'provider': 'DraftKings', 'spread': 8.5, 'fo...",0.279,2.0,1.0,True,True,2,True,notre-dame,ohio-state


In [72]:
manual_fixes = pd.DataFrame([
    {"cfbdata_name": "UL Monroe", "ratings_name": "louisiana-monroe"},
    {"cfbdata_name": "Sam Houston", "ratings_name": "sam-houston-state"},
    {"cfbdata_name": "SE Louisiana", "ratings_name": "southeastern-louisiana"},
    {"cfbdata_name": "Houston Christian", "ratings_name": "houston-baptist"},
    {"cfbdata_name": "UAlbany", "ratings_name": "albany"},
    {"cfbdata_name": "Utah Tech", "ratings_name": "dixiestate"},
])


In [73]:
crosswalk = pd.concat([crosswalk, manual_fixes], ignore_index=True)


In [74]:
games = games.merge(
    crosswalk.rename(columns={"cfbdata_name": "homeTeam", "ratings_name": "ratings_home_team"}),
    on="homeTeam", how="left"
)
games = games.merge(
    crosswalk.rename(columns={"cfbdata_name": "awayTeam", "ratings_name": "ratings_away_team"}),
    on="awayTeam", how="left"
)


In [75]:
missing_home = games[games["ratings_home_team"].isna()]
missing_away = games[games["ratings_away_team"].isna()]

print(f"🔍 Missing home team ratings name: {len(missing_home)}")
print(f"🔍 Missing away team ratings name: {len(missing_away)}")
print("⚠️ Home teams missing crosswalk match:")
print(missing_home["homeTeam"].value_counts().head(10))

print("\n⚠️ Away teams missing crosswalk match:")
print(missing_away["awayTeam"].value_counts().head(10))


🔍 Missing home team ratings name: 0
🔍 Missing away team ratings name: 23
⚠️ Home teams missing crosswalk match:
Series([], Name: count, dtype: int64)

⚠️ Away teams missing crosswalk match:
awayTeam
Savannah St       14
East Texas A&M     3
Merrimack          3
Valparaiso         1
Sacred Heart       1
Lindenwood         1
Name: count, dtype: int64


In [76]:
missing_away = games[games["ratings_away_team"].isna()]
missing_away.groupby("awayTeam")["season"].value_counts().unstack().fillna(0).astype(int)


season,2012,2013,2014,2015,2016,2017,2018,2022,2023,2024
awayTeam,,,,,,,,,,
East Texas A&M,1,0,0,0,0,0,0,0,1,1
Lindenwood,0,0,0,0,0,0,0,0,0,1
Merrimack,0,0,0,0,0,0,0,0,1,2
Sacred Heart,0,0,0,0,0,0,0,0,1,0
Savannah St,2,2,3,2,2,1,2,0,0,0
Valparaiso,0,0,0,0,0,0,0,1,0,0


In [77]:
unmatched_teams = ["East Texas A&M", "Savannah St", "Valparaiso"]

games_with_unmatched = games[
    games["homeTeam"].isin(unmatched_teams) | games["awayTeam"].isin(unmatched_teams)
]
games_with_unmatched[["season", "homeTeam", "awayTeam", "startDate", "venue", "homePoints", "awayPoints"]]


,season,homeTeam,awayTeam,startDate,venue,homePoints,awayPoints
1673,2012,Oklahoma State,Savannah St,2012-09-01T23:00:00.000Z,Boone Pickens Stadium,84.0,0.0
1712,2012,UTSA,East Texas A&M,2012-09-08T18:00:00.000Z,Alamodome,27.0,16.0
1734,2012,Florida State,Savannah St,2012-09-08T22:00:00.000Z,Doak Campbell Stadium,55.0,0.0
2587,2013,Troy,Savannah St,2013-09-07T23:00:00.000Z,Veterans Memorial Stadium (AL),66.0,3.0
2714,2013,Miami,Savannah St,2013-09-21T23:00:00.000Z,Hard Rock Stadium,77.0,7.0
3372,2014,Middle Tennessee,Savannah St,2014-08-30T23:00:00.000Z,"Johnny ""Red"" Floyd Stadium",61.0,7.0
3433,2014,Georgia Southern,Savannah St,2014-09-06T22:00:00.000Z,Allen E. Paulson Stadium,83.0,9.0
4032,2014,BYU,Savannah St,2014-11-22T20:00:00.000Z,LaVell Edwards Stadium,64.0,0.0
4229,2015,Colorado State,Savannah St,2015-09-05T20:00:00.000Z,Sonny Lubick Field at Hughes Stadium,65.0,13.0
4351,2015,Akron,Savannah St,2015-09-19T16:00:00.000Z,InfoCision Stadium,52.0,9.0


In [85]:
games["startDate"] = pd.to_datetime(games["startDate"], utc=True, errors="coerce")
# Default: Eastern Time
games["local_game_date"] = games["startDate"].dt.tz_convert("US/Eastern").dt.date

# Overwrite for Hawaii games
mask_hawaii = games["homeTeam"].str.contains("Hawai", case=False, na=False)
games.loc[mask_hawaii, "local_game_date"] = games.loc[mask_hawaii, "startDate"].dt.tz_convert("Pacific/Honolulu").dt.date



In [87]:
# --- Prep ratings ---
ratings["date"] = pd.to_datetime(ratings["date"]).dt.date
ratings["key1"] = ratings["key1"].astype(str)
ratings["key2"] = ratings["key2"].astype(str)
ratings["team_a"] = ratings[["key1", "key2"]].min(axis=1)
ratings["team_b"] = ratings[["key1", "key2"]].max(axis=1)
# --- Prep games ---
games["ratings_home_team"] = games["ratings_home_team"].astype(str)
games["ratings_away_team"] = games["ratings_away_team"].astype(str)
games["team_a"] = games[["ratings_home_team", "ratings_away_team"]].min(axis=1)
games["team_b"] = games[["ratings_home_team", "ratings_away_team"]].max(axis=1)


In [88]:
games_with_ratings = games.merge(
    ratings[["date", "team_a", "team_b", "Net", "ViewStream"]],
    left_on=["local_game_date", "team_a", "team_b"],
    right_on=["date", "team_a", "team_b"],
    how="left"
)



In [93]:
games_with_ratings["isTelevised"] = games_with_ratings["Net"].notna()

In [94]:
sampled_televised_games = (
    games_with_ratings[games_with_ratings["isTelevised"]]
    .groupby("season")
    .apply(lambda x: x.sample(1, random_state=42) if len(x) > 0 else None)
    .reset_index(drop=True)
)


/var/folders/5n/80tqy79x1w5g7vcylbjsq9cw0000gn/T/ipykernel_50564/2895365962.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  games_with_ratings[games_with_ratings["isTelevised"]]


In [96]:
games_with_ratings.columns

Index(['id', 'season', 'week', 'seasonType', 'startDate', 'startTimeTBD',
       'completed', 'neutralSite', 'conferenceGame', 'attendance', 'venueId',
       'venue', 'homeId', 'homeTeam', 'homeClassification', 'homeConference',
       'homePoints', 'homeLineScores', 'homePostgameWinProbability',
       'homePregameElo', 'homePostgameElo', 'awayId', 'awayTeam',
       'awayClassification', 'awayConference', 'awayPoints', 'awayLineScores',
       'awayPostgameWinProbability', 'awayPregameElo', 'awayPostgameElo',
       'excitementIndex', 'highlights', 'notes', 'mediaType', 'outlet',
       'lines', 'homeWinProbability', 'homeRank', 'awayRank', 'homeRanked',
       'awayRanked', 'rankedTeamCount', 'isRankedMatchup', 'ratings_home_team',
       'ratings_away_team', 'local_game_date', 'team_a', 'team_b', 'date',
       'Net', 'ViewStream', 'isTelevised'],
      dtype='object')

In [99]:
fbs_teams = ['Air Force',
 'Akron',
 'Alabama',
 'App State',
 'Arizona',
 'Arizona State',
 'Arkansas',
 'Arkansas State',
 'Army',
 'Auburn',
 'Ball State',
 'Baylor',
 'Boise State',
 'Boston College',
 'Bowling Green',
 'Buffalo',
 'BYU',
 'California',
 'Central Michigan',
 'Charlotte',
 'Cincinnati',
 'Clemson',
 'Coastal Carolina',
 'Colorado',
 'Colorado State',
 'Duke',
 'East Carolina',
 'Eastern Michigan',
 'Florida',
 'Florida Atlantic',
 'Florida International',
 'Florida State',
 'Fresno State',
 'Georgia',
 'Georgia Southern',
 'Georgia State',
 'Georgia Tech',
 "Hawai'i",
 'Houston',
 'Illinois',
 'Indiana',
 'Iowa',
 'Iowa State',
 'Jacksonville State',
 'James Madison',
 'Kansas',
 'Kansas State',
 'Kennesaw State',
 'Kent State',
 'Kentucky',
 'Liberty',
 'Louisiana',
 'Louisiana Tech',
 'Louisville',
 'LSU',
 'Marshall',
 'Maryland',
 'Massachusetts',
 'Memphis',
 'Miami',
 'Miami (OH)',
 'Michigan',
 'Michigan State',
 'Middle Tennessee',
 'Minnesota',
 'Mississippi State',
 'Missouri',
 'Navy',
 'NC State',
 'Nebraska',
 'Nevada',
 'New Mexico',
 'New Mexico State',
 'North Carolina',
 'Northern Illinois',
 'North Texas',
 'Northwestern',
 'Notre Dame',
 'Ohio',
 'Ohio State',
 'Oklahoma',
 'Oklahoma State',
 'Old Dominion',
 'Ole Miss',
 'Oregon',
 'Oregon State',
 'Penn State',
 'Pittsburgh',
 'Purdue',
 'Rice',
 'Rutgers',
 'Sam Houston',
 'San Diego State',
 'San José State',
 'SMU',
 'South Alabama',
 'South Carolina',
 'Southern Miss',
 'South Florida',
 'Stanford',
 'Syracuse',
 'TCU',
 'Temple',
 'Tennessee',
 'Texas',
 'Texas A&M',
 'Texas State',
 'Texas Tech',
 'Toledo',
 'Troy',
 'Tulane',
 'Tulsa',
 'UAB',
 'UCF',
 'UCLA',
 'UConn',
 'UL Monroe',
 'UNLV',
 'USC',
 'Utah',
 'Utah State',
 'UTEP',
 'UTSA',
 'Vanderbilt',
 'Virginia',
 'Virginia Tech',
 'Wake Forest',
 'Washington',
 'Washington State',
 'Western Kentucky',
 'Western Michigan',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [103]:
games_with_ratings = games_with_ratings[games_with_ratings["seasonType"] != "postseason"]
champ_mask = games_with_ratings["notes"].str.contains("championship", case=False, na=False)
games_with_ratings = games_with_ratings[~champ_mask]

In [130]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Step 1: Filter for valid, televised games
df_tv = games_with_ratings[
    games_with_ratings["isTelevised"] & 
    games_with_ratings["ViewStream"].notna()
].copy()

df_tv["log_viewers"] = np.log(df_tv["ViewStream"])

# Step 2: Lowercase team names
df_tv["team_a"] = df_tv["team_a"].str.lower()
df_tv["team_b"] = df_tv["team_b"].str.lower()

# Step 3: Count games per team
view_counts = pd.concat([
    df_tv[["team_a", "log_viewers"]].rename(columns={"team_a": "team"}),
    df_tv[["team_b", "log_viewers"]].rename(columns={"team_b": "team"})
])
team_game_counts = view_counts.groupby("team").size()

# Step 4: Identify FCS-like teams (<10 appearances)
low_exposure_cutoff = 10
fcs_like_teams = team_game_counts[team_game_counts < low_exposure_cutoff].index.tolist()

# Step 5: Add FCS dummy to df_tv
df_tv["is_fcs_opponent"] = (
    df_tv["team_a"].isin(fcs_like_teams) |
    df_tv["team_b"].isin(fcs_like_teams)
).astype(int)

# Step 6: Create dummy matrix for team_a and team_b
team_dummies_a = pd.get_dummies(df_tv["team_a"])
team_dummies_b = pd.get_dummies(df_tv["team_b"])
X = team_dummies_a.add(team_dummies_b, fill_value=0)

# Step 7: Remove all FCS-like teams from the matrix
X = X.drop(columns=[col for col in X.columns if col in fcs_like_teams])

# Step 8: Compute average log viewers for high-volume FBS teams
valid_fbs_core = team_game_counts[team_game_counts >= 10].index.tolist()
avg_log_views_core = (
    view_counts[view_counts["team"].isin(valid_fbs_core)]
    .groupby("team")["log_viewers"]
    .mean()
    .sort_values()
)

# Step 9: Drop the median team as the reference
median_team = avg_log_views_core.index[len(avg_log_views_core) // 2]
print(f"📎 Dropping high-volume median team as reference: {median_team}")

if median_team in X.columns:
    X = X.drop(columns=[median_team])

# Step 10: Add FCS opponent dummy (row-wise)
X["is_fcs_opponent"] = df_tv["is_fcs_opponent"].values
X = X.astype(float)

# Step 11: Fit model
y = df_tv["log_viewers"]
model = sm.OLS(y.values, X.values).fit()

# Step 12: Extract results
team_effects = pd.Series(
    model.params[:len(X.columns) - 1],  # exclude FCS dummy
    index=X.columns[:-1]
).sort_values(ascending=False)

fcs_effect = model.params[-1]
print(f"📉 Estimated log-viewership penalty for FCS/low-exposure team: {fcs_effect:.3f}")


📎 Dropping high-volume median team as reference: illinois
📉 Estimated log-viewership penalty for FCS/low-exposure team: 5.730


In [131]:
team_effects

ohio-state                8.533808
alabama                   8.230366
michigan                  8.110572
notre-dame                7.998771
penn-state                7.965794
wisconsin                 7.965010
georgia                   7.914087
oklahoma                  7.836982
nebraska                  7.812812
louisiana-state           7.716869
clemson                   7.676986
florida-state             7.662348
florida                   7.637575
texas                     7.634839
oregon                    7.609809
auburn                    7.569134
texas-am                  7.513766
southern-california       7.480452
tennessee                 7.468350
michigan-state            7.457235
arkansas                  7.300584
washington                7.291806
iowa                      7.279724
oklahoma-state            7.267571
south-carolina            7.261175
mississippi               7.225481
miami-fl                  7.195626
louisville                7.156160
navy                

In [129]:
team_game_counts

team
abilene                     1
air-force                  39
akron                      14
alabama                   112
alabama-birmingham          9
appalachian-state          32
arizona                    66
arizona-state              71
arkansas                   64
arkansas-state             27
army                       25
auburn                     90
ball-state                 19
baylor                    101
bethune-cookman             1
boise-state                81
boston-college             42
bowling-green-state        28
brigham-young              92
buffalo                    27
california                 65
campbell                    1
central-florida            76
central-michigan           20
charlotte                   5
chattanooga                 1
cincinnati                 67
clemson                   110
coastal-carolina           12
colorado                   46
colorado-state             23
connecticut                35
delaware                    1
duke 